In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Notes
(Les idées viennent d'une tuto Youtube)

Objectif :
- Comprendre du mieux possible nos données (un petit pas en avant vaut mieux qu'un grand pas en arriere)
- Développer une premiere stratégie de modélisation

### Checklist de base
#### Analyse de Forme :
- **variable target** : 
- **lignes et colonnes** : 
- **types de variables** : 
- **Analyse des valeurs manquantes** :

#### Analyse de Fond :
- **Visualisation de la target** :
- **Signification des variables** :
- **Relation Variables / Target** :

#### Analyse plus détaillée :
- **Relation Variables / Variables** :
- **NaN analyse** :

#### hypotheses nulle (H0) :

In [ ]:
# Idées immo-data

In [ ]:
# Idées century21

In [ ]:
# Idées housedata

## Immo-data

In [3]:
file_path = 'data/immodataParis.csv'
data = pd.read_csv(file_path)

#### Analyse de Forme :

In [5]:
df = data.copy()
df.shape
df.dtypes.value_counts()

int64     5
object    4
dtype: int64

In [7]:
df.head()

,Adresse,Ville,Arrondissement,Type,Prix (€),Prix mensuel (€),Pièce(s),Surface (m2),Date de vente
0,270 RUE SAINT-HONORÉ,PARIS,75001,Appartement,750400,14431,2,52,22/05/2023
1,186 RUE DE RIVOLI,PARIS,75001,Appartement,330000,14348,1,23,28/04/2023
2,23 RUE DE RICHELIEU,PARIS,75001,Appartement,360100,15657,1,23,29/03/2023
3,27 RUE DE RICHELIEU,PARIS,75001,Appartement,286123,11005,1,26,10/02/2023
4,272 RUE SAINT-HONORÉ,PARIS,75001,Appartement,411636,12864,2,32,28/12/2022


In [8]:
df.columns

Index(['Adresse', 'Ville', 'Arrondissement', 'Type', 'Prix (€)',
       'Prix mensuel (€)', 'Pièce(s)', 'Surface (m2)', 'Date de vente'],
      dtype='object')

- **Adresse** : Valeur unique
- **Ville** : Même valeur
- **Arrondissement** : A regouper ?
- **Type** : A regrouper
- **Prix (€)** : A retirer ?
- **Prix mensuel (€)** : Pour la prédiction
- **Pièce(s)** : Sort
- **Surface (m2)** : Sort
- **Date de vente** : Sort

#### Analyse de Fond :

#### Analyse plus détaillée :

#### hypotheses nulle (H0) :